<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Subqueries
              
</p>
</div>

Data Science Cohort Live NYC August 2023
<p>Phase 2</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

Queries can get complex:
- Helpful/necessary to build a query out of sub-units
- These subunits are queries which might include:
    - groupbys/agg functions
    - selecting/filtering
    - etc.

<img src = "Images/sql_subquery.jpg">

Sometimes:

- Query a table.
- Filter/select/... on that query
- Return result of querying on the first query.

First query is a **subquery**.

Inner query is a **subquery**:

- Generates an intermediate table that stores results temporarily in memory.
- Subquery intermediate table can be used during execution of main query.

Subqueries can be used to great effect in a variety of circumstances:
    
- Selecting from a table created by a subquery 
    - Subquery in FROM statement
- Filtering on a subquery.
    - Subquery in a WHERE statement
- Filtering in a SELECT statement.


#### Subquery in FROM clause


First load our handy dandy flights database.

In [1]:
import sqlite3
import numpy as np
import pandas as pd

# connect to flights database
con = sqlite3.connect('data/flights.db')

Recall the various tables in flight database:

In [2]:
%%bash 

sqlite3 data/flights.db
.tables

airlines  airports  routes  


In [3]:
pd.read_sql("""

SELECT * 
FROM airports
LIMIT 2

""", con)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


#### Problem

Get the average altitude of the highest altitude airports in the United States, Russia, and China.

Useful to break this into steps with a subquery.

- Select airport name, city, and altitude for the highest airport in each country
- Then filter this on the country.
- Then get the average of the altitudes of the resultant airports.

Step by step.

In [4]:
pd.read_sql("""

SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports

     GROUP BY country


""", con)

,name,city,country,max_altitude
0,FOB Sharana,Sharan,Afghanistan,7400.0
1,Tirana Rinas,Tirana,Albania,126.0
2,Tamanrasset,Tamanrasset,Algeria,4518.0
3,Fitiuta Airport,Fiti\\'uta,American Samoa,110.0
4,Lubango,Lubango,Angola,5778.0
...,...,...,...,...
235,Jerusalem,Jerusalem,West Bank,2485.0
236,Smara Airport,Smara,Western Sahara,350.0
237,Sanaa Intl,Sanaa,Yemen,7216.0
238,Kasompe,Kasompe,Zambia,4636.0


This is a table produced by the query. We now want to filter this table on country.

- Can select FROM results of above table and filter WHERE conditon is true.
- Subquery creates intermediate table for us.

In [5]:

pd.read_sql("""

SELECT *
    
FROM 

(SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     
     ) AS high_alt_airp
     
 WHERE high_alt_airp.country IN ("United States", "Russia", "China")



""", con)

,name,city,country,max_altitude
0,Yading Daocheng,Daocheng,China,14472.0
1,Irkutsk-2,Irkutsk,Russia,13411.0
2,Silverton,Silverton,United States,9308.0


Now I select the mean altitude across the three countries.

In [6]:
# Average altitude of the highest airports in Russia, United States, and China

pd.read_sql("""

SELECT AVG(high_alt_airp.max_altitude) AS max_altitude_avg
    
FROM 

(SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     
     ) AS high_alt_airp
     
 WHERE high_alt_airp.country IN ("United States", "Russia", "China")



""", con)

,max_altitude_avg
0,12397.0


One can create, in principle, a new table of derived columns and do selections and filtering on that new table.

Let's take a look at our routes table again.

In [7]:
pd.read_sql("""

SELECT * 
     FROM routes
     LIMIT 5


""", con)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


#### Using derived tables in joins

- create set of tables from subqueries
- use these to create a new table with derived columns from subqueries. 

Let's take an example of this.

For each airport I want:
- the count of routes that leaves from there.
- the count of routes that arrive there.

Many ways to do this. But we'll make full use of subqueries and joins here.

In [8]:
departure_table = pd.read_sql("""

SELECT source, COUNT(*) as departure_count
     FROM routes
     
     GROUP BY source
     
""", con)

In [9]:
departure_table

,source,departure_count
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


In [10]:
arrival_table = pd.read_sql("""

SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest
     
""", con)

In [11]:
arrival_table

,dest,arrival_count
0,AAE,9
1,AAL,21
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3413,ZUH,59
3414,ZUM,2
3415,ZVK,3
3416,ZYI,15


We can link these the arrival tables and destination tables together:
- Keep records where source and destination are in both tables.
- What operation?

INNER JOIN ON SUBQUERY RESULTS!

In [12]:

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest
     
""", con)

,airport_code,departure_count,arrival_count
0,AAE,9,9
1,AAL,20,21
2,AAN,2,2
3,AAQ,3,3
4,AAR,8,8
...,...,...,...
3397,ZUH,60,59
3398,ZUM,2,2
3399,ZVK,3,3
3400,ZYI,15,15


This result is a derived table telling us information for each airport and keeping relevant records.

- We can filter or aggregate on this derived table as we see fit.


- Get all airports with significance difference in:
    - number of routes originating and terminating at given airport.
    - dfference in departure and arrival route counts greater than 5.
- Order in descending order of this difference.

Help me out!


In [13]:

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count,
        ABS(dep.departure_count - arriv.arrival_count) AS count_diff
        
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest

     
""", con)

,airport_code,departure_count,arrival_count,count_diff
0,AAE,9,9,0
1,AAL,20,21,1
2,AAN,2,2,0
3,AAQ,3,3,0
4,AAR,8,8,0
...,...,...,...,...
3397,ZUH,60,59,1
3398,ZUM,2,2,0
3399,ZVK,3,3,0
3400,ZYI,15,15,0


<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 


pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count, 
        ABS(dep.departure_count - arrival_count) AS count_diff
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest

WHERE count_diff > 5
ORDER BY count_diff DESC
     
""", con)
```
</details>

**Exercise**

Get me the name, city, country and airport code of airports for countries with more than 10 active airlines companies

Hints:
- DISTINCT(name) will be useful.
- A join might be useful. Aliasing tables will be important here as well.
- Think of groupbys and aggregations and filtering on them.

In [14]:
%%bash 

sqlite3 data/flights.db
.schema airlines

CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
);
CREATE INDEX ix_airlines_index ON airlines ([index]);


In [15]:
%%bash 

sqlite3 data/flights.db
.schema airports

CREATE TABLE airports (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [city] TEXT,
  [country] TEXT,
  [code] TEXT,
  [icao] TEXT,
  [latitude] TEXT,
  [longitude] TEXT,
  [altitude] TEXT,
  [offset] TEXT,
  [dst] TEXT,
  [timezone] TEXT
);
CREATE INDEX ix_airports_index ON airports ([index]);


In [33]:
pd.read_sql("""

SELECT *
FROM airlines
WHERE active = 'Y'


""",con)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
2,9,10,40-Mile Air,\N,Q5,MLA,MILE-AIR,United States,Y
3,12,13,Ansett Australia,\N,AN,AAA,ANSETT,Australia,Y
4,13,14,Abacus International,\N,1B,None,None,Singapore,Y
...,...,...,...,...,...,...,...,...,...
1156,6042,19827,Regionalia Chile,Regionalia Chile,9J,CR1,None,Chile,Y
1157,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y
1158,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
1159,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y


In [28]:
# Get me the name, city, country and airport code of airports for countries with more than 10 active airlines companies

pd.read_sql("""

SELECT name, city, country, code

FROM airports

WHERE country IN (SELECT country
FROM airlines
WHERE active = 'Y'
GROUP BY country
HAVING COUNT(active)>10)
""", con)

# (SELECT airlines.id, COUNT(*) 
# FROM airlines
# GROUP BY country)

# ON airports.country = airlines.country

# """, con)


# pd.read_sql("""
# SELECT name, city, country, code
# FROM airports 
# WHERE country IN (SELECT country
# FROM airlines
# WHERE active = 'Y'
# GROUP BY country
# HAVING COUNT(active) > 10)
# """, con)    

,name,city,country,code
0,Sault Ste Marie,Sault Sainte Marie,Canada,YAM
1,Winnipeg St Andrews,Winnipeg,Canada,YAV
2,Shearwater,Halifax,Canada,YAW
3,St Anthony,St. Anthony,Canada,YAY
4,Tofino,Tofino,Canada,YAZ
...,...,...,...,...
4984,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU
4985,Port McNeill Airport,Port McNeill,Canada,YMP
4986,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG
4987,Deer Harbor Seaplane,Deer Harbor,United States,DHB


<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""
SELECT ar.name, ar.code, ar.city, ar.country, ct
FROM 
( SELECT country, COUNT(DISTINCT name) AS ct
FROM airlines 
WHERE active = 'Y'
GROUP BY country
HAVING ct > 10 ) as inter INNER JOIN airports as ar
ON inter.country = ar.country

""", con)

    
#alternate answer
pd.read_sql("""
SELECT name, city, country, code
FROM airports 
WHERE country IN (SELECT country
FROM airlines
WHERE active = 'Y'
GROUP BY country
HAVING COUNT(active) > 10)
""", con)    
    
```
</details>

- Complex as subqueries pile on. 
- There are ways to organize subqueries: common table expressions
- Will make code readable.

But first: other ways subqueries can be used.

#### Subqueries in the WHERE statement

- Can use the result of a subquery to filter another selection.

I want all route records where the departure airport has a departure route count greater than 200.

In [ ]:
#subquery
pd.read_sql('''


SELECT source, COUNT(source)
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200
''', con)

In [ ]:
pd.read_sql('''


SELECT *
            
FROM routes

WHERE source IN (SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200) 
''', con)

#### The Semijoin

- Using a subquery from one table to filter another table.

Example:

Get the airport name, code, country, and altitude for airports with more than 200 routes originating from that airport.

In [ ]:
pd.read_sql('''

SELECT *
FROM airports

WHERE code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200 
)
''', con)

**Exercise**

Get me routes (all columns) whose destination airport is in the America/New York time zone.

In [ ]:
%%bash

sqlite3 data/flights.db
.schema airports

In [ ]:
%%bash

sqlite3 data/flights.db
.schema airlines

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql('''
SELECT *
FROM routes
WHERE dest IN
(SELECT code 
FROM airports 
WHERE timezone = 'America/New_York')

''', con)
```
</details>

#### The antijoin

Just the negation of the semijoin:

In [ ]:

pd.read_sql('''
SELECT *
FROM airports

WHERE NOT code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200 
)
''', con)

#### Subqueries in the SELECT statement

Get altitude of airport side by side with average airport altitude in the respective country.

In [ ]:

pd.read_sql('''
SELECT a1.country,
name, 
CAST(altitude as int) AS altitude ,

(SELECT AVG(CAST(altitude as int))
FROM airports AS a2
WHERE a2.country == a1.country
) AS avg_alt

FROM airports AS a1


''', con)

Sometimes known as a **correlated subquery**.

- Inner select statement filters a2 by country matching a given row's country in a1. Takes average altitude.

- Does this for each row in a1. The filtering of a2 by country *correlated* with current row in a1.


**Correlated subqueries can be a bit slow**


#### Common Table Expressions (CTEs)

Useful for when:

- Multiple subqueries floating around.
- Reusing same subqueries over and over again in same statement.

Cleans code up, reduces error, makes readable.

CTE syntax:

WITH subquery1_name AS (subquery1 statement),
<br>
WITH subquery2_name AS (subquery2 statement), 
<br>
...., 
<br>
WITH subquery_n_name AS (subquery_n statement)
<br>
<br>
MAIN QUERY

In [ ]:

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest
     
""", con)

In [ ]:

pd.read_sql("""

WITH s1 AS (
     SELECT source, COUNT(source) as departure_count
     FROM routes
     GROUP BY source),
     
     s2 AS ( 
     SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     GROUP BY dest
     )


SELECT source, departure_count, arrival_count
FROM
s1 INNER JOIN s2
ON s1.source = s2.dest

     
""", con)

CTEs make the primary query easy to read.

- Subqueries can just be referenced by looking it up in the CTE.